## 注意事項
要先調整 PROJECT_ROOT = Path('/Users/Jer_ry/Desktop/scripts') 

In [25]:
import os
import inspect
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
import numpy as np
from pathlib import Path
import csv
import random 
import pdb
import os
import pandas as pd
from queue import *
from scipy.stats import poisson
from collections import defaultdict

In [83]:
PROJECT_ROOT = Path('/Users/Jer_ry/Desktop/scripts')   # already defined elsewhere
def generate_social_score(fname='sim1', n_agents=3,
                          MIN=-30, MAX=0, MEAN=-10, SD=10):

    # where to save
    csv_dir  = SIM_ROOT / 'log' / 'social_score' 
    csv_dir.mkdir(parents=True, exist_ok=True)
    out_path = csv_dir / f'{fname}.csv'

    # make the matrix
    raw   = np.random.randint(MIN, MAX + 1, (n_agents, n_agents))
    np.fill_diagonal(raw, 0)

    z     = (raw - raw.mean()) / raw.std()
    score = z * SD + MEAN

    # header row
    labels = [str(i+1) for i in range(n_agents)]
    df = pd.DataFrame(score, columns=labels, index=labels)

    df.to_csv(out_path) 
    print(f'✓ social score saved -> {out_path}')
    return df

## Simulation data dynamic
- 如果要一次跑多個 trial 的話也要回去修改


### 主函式
Parameters:
- FNAME (str): 要讀取的 social score CSV 檔名 （包含 social scores (without extension).
- RANDOM_NUM_GOALS (bool): If True, the number of goals will vary across mazes.
- VERSION_NAME (str): 模擬版本名稱，決定輸出目錄名稱
- STEP_TOTAL (int): 每個迷宮模擬最多執行幾步
- AGENT_NAME (str): 產生的文字檔開頭名稱
- SET_UP_MAZE_TOTAL (int): 要產生幾個迷宮
- PRINT_FINAL_MAZE (bool):是否印出最後一個迷宮狀態的文字檔

Outputs:
- 初始迷宮的 Text files：<AGENT_NAME>_<maze_total>.txt
- 最終迷宮的 Text files: <AGENT_NAME>_<maze_total>_FINAL.txt
- 'summary.csv': 總結模擬結果
    - maze: 第幾次模擬。
    - steps: 最後穩定時的步數。
    - maze_final_binary: 10-bit binary string，表示最後 agent 的鄰近關係。
    - maze_final_decimal: 將上面 binary 轉為 0~1023 的數字。
    - center_agent: 中心 agent 是誰。
    - flag_1 / flag_2: 是否由這個條件終止
- All output files are saved in a directory structured as: current_working_directory/../../new_scripts/data/data_dynamic/<VERSION_NAME>

In [66]:
def Center_Agent(fix_central_index, maze_total, n_agents):
    if fix_central_index:                           
        return fix_central_index[maze_total % len(fix_central_index)]
    else:                                           
        return maze_total % n_agents
## 迷宮初始化 
# 25x25 的格子中間有一個 12x12 的空間區域
def game_art(center_agent, n_move_agents, n_chosen_agents, Chosen_agents_label, Chosen_agents_index):
    """
    初始化迷宮環境，並隨機分配 agent 的初始位置。
    """
    GAME_ART = [
    ['##########################',
     '#                        #',
     '#                        #',
     '#                        #',
     '#                        #',
     '#                        #',  # Environment.
     '#     ##############     #',
     '#     #            #     #',
     '#     #            #     #',
     '#     #            #     #',
     '#     #            #     #',
     '#     #            #     #',
     '#     #            #     #',
     '#     #            #     #',
     '#     #            #     #',
     '#     #            #     #',
     '#     #            #     #',
     '#     #            #     #',
     '#     #            #     #',
     '#     ##############     #',
     '#                        #',
     '#                        #',
     '#                        #',
     '#                        #',
     '#                        #',
     '##########################'],
        ]

    available = [(x,y)
                 for x in range(7,19)
                 for y in range(7,19)
                 if not (x==12 and y==12)]
    # 從裡面一次抽 n_move_agents 個互異的 (x,y)
    chosen = random.sample(available, n_move_agents)
    x_random = [p[0] for p in chosen]
    y_random = [p[1] for p in chosen]
    x_random.insert(center_agent, 12)
    y_random.insert(center_agent, 12)

    for i in range(n_chosen_agents):
        row = GAME_ART[0][y_random[i]]
        GAME_ART[0][y_random[i]] = row[:x_random[i]] \
            + Chosen_agents_label[Chosen_agents_index[i]] \
            + row[x_random[i]+1:]

    return x_random, y_random, GAME_ART
        
# agent start place, distance and energy
def inputs(agent_place, x_random, y_random, social_reward, n_chosen_agents):
    if np.all(agent_place) == 0:
        for i in range(n_chosen_agents):
            agent_place[i] = np.array([x_random[i], y_random[i]])

    distance = np.zeros((n_chosen_agents, n_chosen_agents))
    for i in range(n_chosen_agents):
        for j in range(n_chosen_agents):
            distance[i][j] = np.hypot(agent_place[i][0]-agent_place[j][0],
                                      agent_place[i][1]-agent_place[j][1])

    dist_sq = np.square(distance) + 1e-6        # 加 ε 避免除 0
    force   = social_reward / dist_sq
    return agent_place, distance, force

def final_maze(all_agent_place, Chosen_agents_label, Chosen_agents_index, n_chosen_agents):
    """
    輸出最終迷宮狀態。
    """           #output the final state of the maze (coords need to be added)
    """
    flag_1	所有 agent 都沒有移動
    flag_2	agent 跳回兩步前的狀態（擺動不穩）
    """
    GAME_FINAL = [
    ['##########################',
     '#                        #',
     '#                        #',
     '#                        #',
     '#                        #',
     '#                        #',  # Environment.
     '#     ##############     #',
     '#     #            #     #',
     '#     #            #     #',
     '#     #            #     #',
     '#     #            #     #',
     '#     #            #     #',
     '#     #            #     #',
     '#     #            #     #',
     '#     #            #     #',
     '#     #            #     #',
     '#     #            #     #',
     '#     #            #     #',
     '#     #            #     #',
     '#     ##############     #',
     '#                        #',
     '#                        #',
     '#                        #',
     '#                        #',
     '#                        #',
     '##########################'],
        ]

    final_agent_place = np.zeros((n_chosen_agents,2))
    for i in range(-n_chosen_agents,0):
        final_agent_place[i] = all_agent_place[i,]
    final_agent_place = final_agent_place.astype(int)

    for i in range(n_chosen_agents):
        GAME_FINAL[0][final_agent_place[i,1]]=GAME_FINAL[0][final_agent_place[i,1]][0:final_agent_place[i,0]]+Chosen_agents_label[Chosen_agents_index[i]]+GAME_FINAL[0][final_agent_place[i,1]][final_agent_place[i,0]+1:]
    
    
    return GAME_FINAL

In [67]:
DIR8 = [(-1,-1), (0,-1), (1,-1),
        (-1, 0),          (1, 0),
        (-1, 1), (0, 1), (1, 1)]
def move(agent_place, force, n_chosen_agents):
    """
    1) 計算每個 agent 的合力向量 force_vector_sum
    2) 根據合力方向與閾值 (>0.5) 決定擬定移動方向 (-1/0/1)
    3) 擬定所有新位置（同時夾在內室 [7,18] 之內）
    4) 如果一個位置只有一個 agent 想去，直接移動；
       如果多個想去，挑合力最大者移動，剩下的人都留原地。
    """
    # （1）計算 force_vector_sum
    force_vector = np.zeros((n_chosen_agents, 2 * n_chosen_agents))
    for i in range(n_chosen_agents):
        for j in range(n_chosen_agents):
            if i == j: continue
            dx = agent_place[j,0] - agent_place[i,0]
            dy = agent_place[j,1] - agent_place[i,1]
            dist = np.hypot(dx, dy)
            if dist > 0:
                f = force[i,j] / (dist**2 + 1e-6)
                force_vector[i,2*j  ] = (dx/dist) * f
                force_vector[i,2*j+1] = (dy/dist) * f

    force_vector_sum = force_vector.reshape(n_chosen_agents, n_chosen_agents, 2).sum(axis=1)

    # （2）從合力向量決定離散移動向量 move_vector
    move_vector = np.zeros_like(force_vector_sum, dtype=int)
    norms = np.linalg.norm(force_vector_sum, axis=1)
    for i in range(n_chosen_agents):
        ux, uy = force_vector_sum[i] / (norms[i] if norms[i]>0 else 1)
        move_vector[i,0] =  1 if ux >=  0.5 else (-1 if ux <= -0.5 else 0)
        move_vector[i,1] =  1 if uy >=  0.5 else (-1 if uy <= -0.5 else 0)

    # （3）擬定所有新位置，並夾在內室 [7,18]
    proposed = []
    for i in range(n_chosen_agents):
        x0, y0 = agent_place[i]
        x1 = min(18, max(7, x0 + move_vector[i,0]))
        y1 = min(18, max(7, y0 + move_vector[i,1]))
        proposed.append((x1, y1))

    # （4）衝突解決：同一個 cell 出現多於 1 次，留在原地；但選一個「力道最強者」破障
    buckets = defaultdict(list)
    for i, pos in enumerate(proposed):
        buckets[pos].append(i)

    new_place = agent_place.copy()
    for pos, idxs in buckets.items():
        if len(idxs) == 1:
            # 無人衝突，直接移動
            i = idxs[0]
            new_place[i] = pos
        else:
            # 衝突：挑合力最大者移動，其他人留原地
            strengths = [norms[i] for i in idxs]
            winner = idxs[np.argmax(strengths)]
            new_place[winner] = pos
            # 其餘的 idxs 不變（留在 agent_place）

    return new_place

def move_random(agent_place, n_chosen_agents, n_sim_agents_for_poisson_mu):
    agent_place_current_step_start = np.array(agent_place, dtype=float)
    agent_place_proposed_updates = np.array(agent_place, dtype=float)

    mu_val = n_sim_agents_for_poisson_mu / 2
    num_moving = 0
    if n_chosen_agents > 0:
        num_moving = min(max(1, poisson.rvs(mu=mu_val)), n_chosen_agents)
    
    if num_moving == 0:
        return agent_place_current_step_start

    moving_agent_indices = np.random.choice(n_chosen_agents, num_moving, replace=False)

    for agent_idx in moving_agent_indices:
        direction = np.random.randint(0, 4)
  
        current_x, current_y = agent_place_current_step_start[agent_idx]
        
        proposed_x, proposed_y = current_x, current_y
        if direction == 0:    # 上
            proposed_y -= 1
        elif direction == 1:  # 右
            proposed_x += 1
        elif direction == 2:  # 下
            proposed_y += 1
        else:                 # 左
            proposed_x -= 1
        
        clamped_x = int(round(min(18, max(7, proposed_x))))
        clamped_y = int(round(min(18, max(7, proposed_y))))

        is_target_center = (clamped_x == 12 and clamped_y == 12)
        
        if not is_target_center:
            if not (clamped_x == current_x and clamped_y == current_y) :
                 agent_place_proposed_updates[agent_idx] = [clamped_x, clamped_y]
    final_agent_positions_this_step = np.array(agent_place_proposed_updates) 

    for i in range(n_chosen_agents):
        for j in range(i + 1, n_chosen_agents):
            if final_agent_positions_this_step[i,0] == final_agent_positions_this_step[j,0] and \
               final_agent_positions_this_step[i,1] == final_agent_positions_this_step[j,1]:
                final_agent_positions_this_step[i] = agent_place_current_step_start[i]
                final_agent_positions_this_step[j] = agent_place_current_step_start[j]
    
    return final_agent_positions_this_step  

# def move_logic_random(positions):
    """
    根據指定的邏輯規則計算三個 agent 的下一步位置。
    1. 隨機選定一個 agent 上下移動，另外兩個左右移動。
    2. 左右移動的 agent 隨機選擇方向 (左/右)。
    3. 如果左右移動方向相同，上下 agent 向下；若方向相反，則向上。

    Args:
        positions (np.array): 形狀為 (3, 2) 的陣列，代表 A, B, C 的當前 (x,y) 座標。

    Returns:
        np.array: 形狀為 (3, 2) 的陣列，代表 agent 的新位置。
    """
    if positions.shape != (3, 2):
        raise ValueError("此邏輯只適用於 3 個 agents。")

    new_positions = positions.copy()
    agent_indices = np.array([0, 1, 2]) # 代表 A, B, C

    # 1. 隨機選出上下和左右移動的 agent
    np.random.shuffle(agent_indices)
    up_down_idx = agent_indices[0]
    left_right_indices = agent_indices[1:]

    # 2. 決定左右移動的方向
    # -1 代表向左, +1 代表向右
    direction_choices = [-1, 1]
    dx1 = random.choice(direction_choices)
    dx2 = random.choice(direction_choices)

    # 更新左右移動的 agent 的 x 座標
    new_positions[left_right_indices[0], 0] += dx1
    new_positions[left_right_indices[1], 0] += dx2

    # 3. 根據左右移動的方向，決定上下 agent 的移動方向
    # 註：假設螢幕座標系，y 增加是向下，y 減少是向上
    if dx1 == dx2:  # 方向相同 (都左或都右)
        # 向下移動
        new_positions[up_down_idx, 1] += 1
    else:  # 方向相反 (一左一右)
        # 向上移動
        new_positions[up_down_idx, 1] -= 1

    # 邊界限制：確保所有 agent 都在 [7, 18] 的活動範圍內
    new_positions = np.clip(new_positions, 7, 18)

    return new_positions

def move_logic(positions, direc_case):
    if positions.shape != (3, 2):
        raise ValueError("此邏輯只適用於 3 個 agents。")

    new_positions = positions.copy()
    direc_rule = []
    if   (direc_case == 1 ):  direc_rule = ['L', 'L', 'D']
    elif (direc_case == 2 ):  direc_rule = ['D', 'L', 'L']
    elif (direc_case == 3 ):  direc_rule = ['L', 'D', 'L']
    elif (direc_case == 4 ):  direc_rule = ['R', 'R', 'D']
    elif (direc_case == 5 ):  direc_rule = ['D', 'R', 'R']
    elif (direc_case == 6 ):  direc_rule = ['R', 'D', 'R']
    elif (direc_case == 7 ):  direc_rule = ['L', 'R', 'U']
    elif (direc_case == 8 ):  direc_rule = ['U', 'L', 'R']
    elif (direc_case == 9 ):  direc_rule = ['U', 'R', 'L']
    elif (direc_case == 10): direc_rule = ['L', 'U', 'R']
    elif (direc_case == 11): direc_rule = ['R', 'U', 'L']
    elif (direc_case == 12): direc_rule = ['R', 'L', 'U']

    for i, dir in enumerate(direc_rule):
        if  (dir == 'L'):  new_positions [i, 0] -= 1
        elif(dir == 'R'):  new_positions [i, 0] += 1
        elif(dir == 'U'):  new_positions [i, 1] += 1
        elif(dir == 'D'):  new_positions [i, 1] -= 1

    # BC
    new_positions = np.clip(new_positions, 7, 18)

    return new_positions

def move_intermediate(pos, primary_idx, case_type):
    """
    Parameters
    ----------
    pos : np.ndarray, shape (3,2)
        目前 A,B,C  (x,y)，必須為 float/int
    primary_idx : tuple(int,int)
        length 2，指出哪兩隻是「主移動」agent 的 idx (0,1,2)(let to be A,B)
    case_type : int
        1  -> C (index 剩下那隻) 走平均位移
        2  -> C 走到 (A',B') 的中點

    Returns
    -------
    new_pos : np.ndarray, shape (3,2)
    """
    if pos.shape != (3,2):
        raise ValueError('move_intermediate only s agent')
    if len(primary_idx) != 2 or any(i not in (0,1,2) for i in primary_idx):
        raise ValueError('primary_idx must be two of 1,2,3')

    sec_idx = 3 - sum(primary_idx)          # 0+1+2 = 3
    new_pos = pos.astype(float).copy()

    # --- A,B ---
    dA = np.array(random.choice(DIR8))
    dB = np.array(random.choice(DIR8))

    new_pos[primary_idx[0]] += dA
    new_pos[primary_idx[1]] += dB

    # --- C ---
    if case_type == 1:
        # 取兩位移平均，再 round 到最近integer
        dC = np.round((dA + dB) / 2).astype(int)
        new_pos[sec_idx] += dC
    elif case_type == 2:
        target = np.round((new_pos[primary_idx[0]] +
                           new_pos[primary_idx[1]]) / 2).astype(int)
        new_pos[sec_idx] = target
    else:
        raise ValueError('case_type 1 or 2')

    # ---- BC ----
    new_pos[:,0] = np.clip(new_pos[:,0], 7, 18) 
    new_pos[:,1] = np.clip(new_pos[:,1], 7, 18) 
    return new_pos

In [68]:
def simulation_data_dynamic(fname_csv='dS103_test',
                            RANDOM_NUM_GOALS=False,
                            VERSION_NAME='sim1',
                            STEP_TOTAL=20,
                            AGENT_NAME="dS_test",
                            N_AGENTS=3,
                            SET_UP_MAZE_TOTAL=1000,
                            PRINT_FINAL_MAZE=False,
                            move_type = 'normal'
                            ):
    move_type = move_type.lower()
    # - dir
    DIR_TXT_OUTPUT   = SIM_ROOT / move_type          # rulemap / random / intermediate
    DIR_TXT_OUTPUT.mkdir(parents=True, exist_ok=True)
    dir_txt   = SIM_ROOT / move_type
    dir_txt.mkdir(parents=True, exist_ok=True)
    file_summary = dir_txt / 'summary.csv'
    FILE_CSV_SUMMARY = DIR_TXT_OUTPUT / 'summary.csv'

    dir_csv    = SIM_ROOT / 'log' / 'social_score' 
    csv_file   = dir_csv / f'{fname_csv}.csv'
    if not csv_file.exists():
        raise FileNotFoundError(csv_file)
    DIR_CSV_OUTPUT = SIM_ROOT / 'log' / 'social_score'
    DIR_CSV_OUTPUT.mkdir(parents=True, exist_ok=True)
    
    agents_arr = pd.read_csv(csv_file, index_col=0).to_numpy(float)

    n_chosen_agents =  N_AGENTS
    n_move_agents = n_chosen_agents-1
    Chosen_agents_index = list(range(n_chosen_agents))
    Chosen_agents_label = (
        ['S'] + [chr(64 + i) for i in range(1, n_chosen_agents)]
    )
    #center_agent= 0  #index of the pinned-at-center agent


    #social reward
    social_reward = agents_arr.copy()
            
    ## social reward check
    # 如果某 agent 對所有人吸引力總和為 0，
    # 代表不會主動移動，
    # 因此將其固定在中心
    fix_central_index=[]
    for i in range (n_chosen_agents):
        check=sum(abs(social_reward[i]))
        if check == 0:
            fix_central_index.append(i)
    maze_total = 0
    df_collect_summary = pd.DataFrame()

    while maze_total < SET_UP_MAZE_TOTAL:
        #new maze
        step=0
        true_step=999
        agent_place = np.zeros((n_chosen_agents,2))                      #recent agent place
        all_agent_place = np.zeros((1,2))                  #all coordinates of agent place
        flag_1=0
        flag_2=0
        
        center_agent=Center_Agent(fix_central_index, maze_total, n_chosen_agents)
        x_random,y_random,GAME_ART=game_art(center_agent, n_move_agents, 
                                            n_chosen_agents, Chosen_agents_label, Chosen_agents_index)
        agent_place = np.zeros((n_chosen_agents, 2))
        agent_place, _, _ = inputs(agent_place,
                              x_random, y_random,
                               social_reward, n_chosen_agents)
        all_agent_place = [agent_place.copy()]   
        same_counter = wobble_counter = 0
        prev_prev    = None
        step = 0

        # for itermediate:
        ## case_type 為 1or 2 random 選一個
        case_type_maze  = random.choice([1, 2])
        primary_idx_maze = tuple(sorted(random.sample(range(3), 2)))
        while step < STEP_TOTAL:
            agent_place, _, force = inputs(agent_place, x_random, y_random,
                        social_reward, n_chosen_agents)
            if move_type == 'rulemap':
                agent_place_new = move(agent_place, force, n_chosen_agents)
            elif move_type == 'random':
                agent_place_new = move_random(agent_place, n_chosen_agents, n_chosen_agents)
            elif move_type == 'intermediate':
                agent_place_new = move_intermediate(
                    agent_place,
                    primary_idx_maze,
                    case_type_maze
                )               
            else:
                raise ValueError(f"unknown move_type: {move_type}")
            all_agent_place.append(agent_place_new.copy())
            step += 1

            same_counter   = same_counter + 1 if np.array_equal(agent_place_new, agent_place) else 0
            wobble_counter = wobble_counter + 1 if prev_prev is not None and \
                            np.array_equal(agent_place_new, prev_prev) else 0

            if same_counter >= 2 or wobble_counter >= 2:
                break

            prev_prev   = agent_place.copy()
            agent_place = agent_place_new.copy()

        true_step = step
        all_agent_place = np.vstack(all_agent_place)       # shape (true_step+1, 5, 2)

                


        #print the last maze here
        GAME_FINAL = final_maze(all_agent_place, Chosen_agents_label,
                                    Chosen_agents_index, n_chosen_agents)
                
        #to fit the model, coordinates are 0~12
        all_agent_place = all_agent_place - 6


        maze_final_stat=np.zeros((n_chosen_agents,n_chosen_agents))
        for i  in range(-n_chosen_agents,0):
            for j in range(-n_chosen_agents,0):
                if(int(all_agent_place[i][1])-int(all_agent_place[j][1]))**2+(int(all_agent_place[i][0])-int(all_agent_place[j][0]))**2 <= 2:
                    maze_final_stat[i][j]=1
                

        #take 10 units of the 5*5 to represent status
        pair_cnt = n_chosen_agents * (n_chosen_agents - 1) // 2
        maze_final_stat_bin = np.zeros(pair_cnt)
        l=0
        for i in range(n_chosen_agents):        # 這裡不用負索引
            for j in range(i):
                maze_final_stat_bin[l] = maze_final_stat[j][i]
                l += 1

        #change into string
        maze_final_stat_bin=maze_final_stat_bin.astype(int)
        maze_final_stat_bin=maze_final_stat_bin.astype(str)
        maze_final_stat_bin="".join(maze_final_stat_bin)

        #bin to dec(0~1023)
        maze_final_stat_dec=int(maze_final_stat_bin,base=2)
        
        maze_final_stat_bin = maze_final_stat_bin.split()
        df_summary = pd.DataFrame({
            "maze"      : [maze_total + 1],
            "steps"     : [true_step],
            "maze_final_binary" : [maze_final_stat_bin],
            "maze_final_decimal": [maze_final_stat_dec],
            "center_agent"      : [Chosen_agents_label[center_agent]],
            "flag_1"            : [flag_1],
            "flag_2"            : [flag_2],
            "case_type"         : [case_type_maze if move_type == "intermediate" else None],
            "move_dominant_agents": [primary_idx_maze if move_type == "intermediate" else None]
        })
        
        # df_collect_summary = df_collect_summary.append(df_summary)
        df_collect_summary = pd.concat([df_collect_summary, df_summary], ignore_index=True)
        try:
            #print start maze
            output_file  = os.path.join(DIR_TXT_OUTPUT, \
                                                str(AGENT_NAME)+"_"+str(maze_total+1)+".txt")
            text_file = open(output_file, "w")
            text_file.write('Maze:\n')
                        
            for i in range(len(GAME_ART[0])):
                text_file.write(GAME_ART[0][i])
                text_file.write('\n')
            np.set_printoptions(threshold=np.inf, linewidth=200)  # 無限長度、不自動換行
            agent_place_str = np.array2string(all_agent_place, separator=' ')
            text_file.write(agent_place_str)
            text_file.write('\n')
            text_file.close()
            # print(GAME_ART) # 印出 maze
            #print end maze
            if PRINT_FINAL_MAZE == True:
                output_file  = os.path.join(DIR_TXT_OUTPUT, \
                                                str(AGENT_NAME)+"_"+str(maze_total+1)+"FINAL.txt")
                text_file = open(output_file, "w")
                text_file.write('Final_Maze:\n')
                # print(GAME_FINAL)
                        
                for i in range(len(GAME_ART[0])):
                    text_file.write(GAME_FINAL[0][i])
                    text_file.write('\n')
                text_file.write('\n')
                text_file.close()
            else:pass
        except:
            print('error')
        maze_total+=1
        if maze_total == SET_UP_MAZE_TOTAL:
            df_collect_summary.to_csv(FILE_CSV_SUMMARY)

def get_parity_map(case_type):
    """ 根據 case_type 返回 A, B, C 的奇偶性映射。"""
    # agent 索引: 0=A, 1=B, 2=C
    if (case_type - 1) // 12 + 1 == 1:
        return {0: 'odd', 1: 'odd', 2: 'even'}
    elif (case_type - 1) // 12 + 1 == 2:
        return {0: 'odd', 1: 'even', 2: 'odd'}
    elif (case_type - 1) // 12 + 1 == 3:
        return {0: 'even', 1: 'odd', 2: 'odd'}
    else:
        raise ValueError("case_type 必須是 1, 2, 或 3。")
    
def run_logic_simulation(case_type, trial_id, total_steps=10, version_name='N3_logic'):
    print(f"--- 開始模擬: Case {case_type}, Trial {trial_id} ---")
    # agent_indices = np.array([0, 1, 2]) 
    # np.random.shuffle(agent_indices)
    # direction_choices = [-1, 1]
    # dx1 = random.choice(direction_choices)
    # dx2 = random.choice(direction_choices)
    direc_case =  case_type % 12
    # 建立輸出目錄
    output_dir = SIM_ROOT / 'logic' 
    output_dir.mkdir(parents=True, exist_ok=True)
    # 初始隨機位置
    initial_positions = np.random.randint(7, 19, size=(3, 2)).astype(float)
    
    trajectory = [initial_positions]
    current_positions = initial_positions.copy()
    
    for step in range(total_steps):
        next_positions = move_logic(current_positions, direc_case)
        trajectory.append(next_positions)
        current_positions = next_positions

    # 將軌跡轉換為 NumPy 陣列並正規化 (減去偏移量)
    trajectory_np = np.stack(trajectory, axis=0)
    trajectory_normalized = trajectory_np - 6.0 # 存檔前正規化

    # 儲存到 TXT 檔案
    # 檔名包含 case_type 以便繪圖時解析
    fname = f"logic_case{case_type}.txt"
    output_path = os.path.join(output_dir, fname)
    
    with open(output_path, 'w') as f:
        f.write("Maze:\n")
        maze_layout = ['##########################'] + ['#                        #']*4 + ['#     ##############     #'] + ['#     #            #     #']*13 + ['#     ##############     #'] + ['#                        #']*5 + ['##########################']
        for row in maze_layout:
            f.write(row + '\n')
        
        np.set_printoptions(threshold=np.inf, linewidth=200)
        agent_place_str = np.array2string(trajectory_normalized, separator=' ')
        f.write(agent_place_str + '\n')
        
    print(f"✓ 模擬完成，檔案儲存於: {output_path}\n")


# 主程式區
### 參數調整

In [88]:
# 用以生成社交網路的參數，
# 使用了 .src.scripts.simulation_data_generator.dynamic.generate_social_score
# 可以生成 5x 6 的矩陣
n_chosen_agents = 3
move_type = 'Random' # RuleMap, Random, Intermediate

FNAME = 'sim1'  # File name for the output CSV file
MIN = -10  # Minimum value for social score
MAX = 10  # Maximum value for social score
MEAN = 1  # Mean value for social score
SD = 2.5  # Standard deviation for social score

# Define constants for the simulation data generation
RANDOM_NUM_GOALS = False  # If True, the number of goals will vary across mazes
VERSION_NAME = FNAME  # Version name used for output directory naming
STEP_TOTAL = 15  # 總共走幾步
AGENT_NAME = FNAME
SET_UP_MAZE_TOTAL = 20  # 總共要畫幾個不同的 maze
PRINT_FINAL_MAZE = False  # Flag to indicate whether to print the final maze

# Define constants for the TOMNet model
LIST_SUBJECTS = FNAME  # List of subject names, should correspond to VERSION_NAMEs
PROJECT_ROOT = Path('/Users/Jer_ry/Desktop/scripts')
SIM_ROOT     = PROJECT_ROOT / 'data' / FNAME 
SIM_ROOT.mkdir(parents=True, exist_ok=True)

### 生成 social score

In [85]:
generate_social_score(FNAME, n_chosen_agents, MIN, MAX, MEAN, SD)

✓ social score saved -> /Users/Jer_ry/Desktop/scripts/data/sim1/log/social_score/sim1.csv


,1,2,3
1,2.521452,-2.042903,-1.586468
2,0.239274,2.521452,-0.217161
3,6.172935,-1.130032,2.521452


In [90]:
simulation_data_dynamic(
    fname_csv=FNAME,
    RANDOM_NUM_GOALS=RANDOM_NUM_GOALS,
    VERSION_NAME=VERSION_NAME,
    STEP_TOTAL=STEP_TOTAL,
    AGENT_NAME=AGENT_NAME,
    N_AGENTS=n_chosen_agents,
    SET_UP_MAZE_TOTAL=SET_UP_MAZE_TOTAL,
    PRINT_FINAL_MAZE=PRINT_FINAL_MAZE,
    move_type = move_type
)

In [72]:
# 專門 for logic simulation 的函式
# 有 36 種 case_type
for CASE_TO_RUN in range(1, 21): # 先取前 20 個 case_type
    SIMULATION_STEPS = 15        # 每個試驗模擬幾步
    run_logic_simulation(
        case_type = CASE_TO_RUN,
        trial_id  = 1, 
        total_steps = SIMULATION_STEPS,
        version_name = VERSION_NAME
    )

--- 開始模擬: Case 1, Trial 1 ---
✓ 模擬完成，檔案儲存於: /Users/Jer_ry/Desktop/scripts/data/sim1/logic/logic_case1.txt

--- 開始模擬: Case 2, Trial 1 ---
✓ 模擬完成，檔案儲存於: /Users/Jer_ry/Desktop/scripts/data/sim1/logic/logic_case2.txt

--- 開始模擬: Case 3, Trial 1 ---
✓ 模擬完成，檔案儲存於: /Users/Jer_ry/Desktop/scripts/data/sim1/logic/logic_case3.txt

--- 開始模擬: Case 4, Trial 1 ---
✓ 模擬完成，檔案儲存於: /Users/Jer_ry/Desktop/scripts/data/sim1/logic/logic_case4.txt

--- 開始模擬: Case 5, Trial 1 ---
✓ 模擬完成，檔案儲存於: /Users/Jer_ry/Desktop/scripts/data/sim1/logic/logic_case5.txt

--- 開始模擬: Case 6, Trial 1 ---
✓ 模擬完成，檔案儲存於: /Users/Jer_ry/Desktop/scripts/data/sim1/logic/logic_case6.txt

--- 開始模擬: Case 7, Trial 1 ---
✓ 模擬完成，檔案儲存於: /Users/Jer_ry/Desktop/scripts/data/sim1/logic/logic_case7.txt

--- 開始模擬: Case 8, Trial 1 ---
✓ 模擬完成，檔案儲存於: /Users/Jer_ry/Desktop/scripts/data/sim1/logic/logic_case8.txt

--- 開始模擬: Case 9, Trial 1 ---
✓ 模擬完成，檔案儲存於: /Users/Jer_ry/Desktop/scripts/data/sim1/logic/logic_case9.txt

--- 開始模擬: Case 10, Trial 1 ---
✓ 模擬完成

## 畫圖圖

In [ ]:
import os, re, math, itertools, random, glob
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
Chosen_agents_label = ['S'] + [chr(64 + i) for i in range(1, n_chosen_agents)]
N_AGENTS = len(Chosen_agents_label)

DIR_ROOT = os.getcwd() 
VERSION_NAME = 'N3'=
AGENT_NAME_PATTERN = "N3" 

# DIR_TXT_OUTPUT 在 __main__ 中會被重新賦值為測試路徑
# 這裡定義一個預設的，以防模組被其他方式導入使用
DIR_TXT_OUTPUT = os.path.join(
        SIM_ROOT,               # ← 換掉 os.getcwd() ...
        VERSION_NAME,           # dS_test 之類
        move_type               # rulemap / random / intermediate
)
STEP_TOTAL = 10 
COORDINATE_OFFSET = 6 
def parse_coordinates(coord_lines_raw):
    """
    從原始座標行列表中解析座標。
    """
    coordinates = []
    for line in coord_lines_raw:
        cleaned_line = line.strip().replace('[', '').replace(']', '')
        parts = cleaned_line.split()
        if len(parts) == 2:
            try:
                coordinates.append([float(parts[0]), float(parts[1])])
            except ValueError:
                print(f"警告：無法解析座標行: {line.strip()}")
    return np.array(coordinates)

def draw_maze_state(base_maze_layout, agent_positions_txt, agent_labels, step_num, maze_filename):
    """
    繪製給定步驟的迷宮狀態。
    :param base_maze_layout: list of strings, 迷宮的基礎佈局
    :param agent_positions_txt: numpy array, shape (N_AGENTS, 2), 從 TXT 讀取的當前步驟 agent 座標
    :param agent_labels: list of strings, agent 的標籤
    :param step_num: int, 當前的步驟編號
    :param maze_filename: str, 正在處理的迷宮檔案名稱
    """
    print(f"\n--- 繪製迷宮: {maze_filename}, 步驟: {step_num} ---")
    
    current_maze = [list(row) for row in base_maze_layout]
    
    for i, label in enumerate(agent_labels):
        if i < len(agent_positions_txt):
            # 從 TXT 讀取的座標需要加上 OFFSET 才是原始的繪圖座標
            x_txt, y_txt = agent_positions_txt[i, 0], agent_positions_txt[i, 1]
            
            # 轉換回繪圖座標
            x_plot = int(round(x_txt + COORDINATE_OFFSET))
            y_plot = int(round(y_txt + COORDINATE_OFFSET))
            
            # 邊界檢查 (使用繪圖座標)
            if 0 <= y_plot < len(current_maze) and 0 <= x_plot < len(current_maze[y_plot]):
                current_maze[y_plot][x_plot] = label
            else:
                print(f"警告：Agent {label} 的繪圖座標 ({x_plot},{y_plot}) 超出迷宮邊界。原始TXT座標: ({x_txt},{y_txt})")
        else:
            print(f"警告：Agent {label} 的座標缺失。")

    for row in current_maze:
        print("".join(row))

def process_maze_file(filepath):
    """
    讀取單個迷宮檔案並繪製所有步驟。
    """
    print(f"處理檔案: {filepath}")
    base_maze_layout = []
    raw_coord_lines = []
    
    try:
        with open(filepath, 'r') as f:
            lines = f.readlines()

        if not lines[0].strip() == "Maze:":
            print(f"錯誤：檔案 {filepath} 的格式不正確，開頭不是 'Maze:'。")
            return
        
        maze_layout_lines = 26 
        base_maze_layout = [line.strip('\n') for line in lines[1:1 + maze_layout_lines]]
        if len(base_maze_layout) != maze_layout_lines:
            print(f"錯誤：檔案 {filepath} 的迷宮佈局行數不足。")
            return
            
        coord_block_str = "".join(lines[1 + maze_layout_lines:]).strip()
        
        if not coord_block_str.startswith("[[") or not coord_block_str.endswith("]]"):
            print(f"警告: 檔案 {filepath} 的座標區塊格式可能不完全符合預期的 [[...]] 包裹。嘗試按行解析。")
            raw_coord_lines = [line.strip() for line in lines[1 + maze_layout_lines:] if line.strip() and '[' in line and ']' in line]
            if not raw_coord_lines:
                print(f"錯誤: 檔案 {filepath} 未能從行中解析出任何座標。")
                return
        else: 
            coord_block_str = coord_block_str[2:-2]
            individual_coord_strs = re.split(r'\]\s*\[', coord_block_str)
            raw_coord_lines = [f"[{s.strip()}]" for s in individual_coord_strs if s.strip()]

        all_agent_coords_parsed = parse_coordinates(raw_coord_lines)
        
        expected_coords_len = (STEP_TOTAL + 1) * N_AGENTS
        if len(all_agent_coords_parsed) != expected_coords_len:
            print(f"警告：檔案 {filepath} 中解析得到的座標數量 ({len(all_agent_coords_parsed)}) 與預期 ({expected_coords_len}) 不符。")
            if len(all_agent_coords_parsed) < N_AGENTS:
                 print(f"錯誤：座標數量不足以繪製任何步驟。")
                 return

        num_frames_to_draw = STEP_TOTAL + 1
        
        for step in range(num_frames_to_draw):
            start_index = step * N_AGENTS
            end_index = (step + 1) * N_AGENTS
            
            if end_index <= len(all_agent_coords_parsed):
                current_step_positions_txt = all_agent_coords_parsed[start_index:end_index]
                if len(current_step_positions_txt) == N_AGENTS:
                     draw_maze_state(base_maze_layout, current_step_positions_txt, Chosen_agents_label, step, os.path.basename(filepath))
                else:
                    print(f"警告：步驟 {step} 的座標數量不足 ({len(current_step_positions_txt)}/{N_AGENTS})。")
                    break 
            else:
                print(f"警告：步驟 {step} 的座標數據不足，無法繪製。")
                break 

    except FileNotFoundError:
        print(f"錯誤：找不到檔案 {filepath}")
    except Exception as e:
        print(f"處理檔案 {filepath} 時發生錯誤: {e}")

def main():
    # DIR_TXT_OUTPUT 會在 if __name__ == '__main__' 區塊中被正確設定 (可能為測試路徑)
    if not os.path.exists(DIR_TXT_OUTPUT):
        print(f"錯誤：找不到目錄 {DIR_TXT_OUTPUT}")
        return

    for filename in os.listdir(DIR_TXT_OUTPUT):
        if filename.startswith(AGENT_NAME_PATTERN) and filename.endswith(".txt") and "FINAL" not in filename:
            filepath = os.path.join(DIR_TXT_OUTPUT, filename)
            process_maze_file(filepath)

#### ACSii 版本
用以簡單測試規則，可忽略

In [ ]:
PROJECT_ROOT   = Path('/Users/Jer_ry/Desktop/scripts')
SIM_FOLDER     = FNAME                    # e.g. 'sim1'
RUN_TYPE       = 'random' 

DIR_TXT        = PROJECT_ROOT / 'data' / SIM_FOLDER / RUN_TYPE
AGENT_PREFIX   = 'sim1_'             # ← 只抓 logic_case*.txt；要全抓就設成 ''
N_AGENTS       = 3
FILES_TO_SHOW  = 1                        # None / int / ['logic_case1_trial_3.txt', ...]
STEPS_TO_SHOW  = 10                       # None / int / [0,2,5]
COORD_OFFSET   = 6
AGENT_LABELS   = ['S'] + [chr(65+i) for i in range(N_AGENTS-1)]
def parse_coords(raw_lines):
    pts=[]
    for ln in raw_lines:
        ln=ln.strip().strip('[]')
        if ln:
            pts.append(tuple(map(float,ln.split())))
    return np.asarray(pts,dtype=float)

def draw_ascii(base_maze,xy,step,fname):
    canvas=[list(r) for r in base_maze]
    for idx,(x_txt,y_txt) in enumerate(xy):
        x=int(round(x_txt+COORD_OFFSET))
        y=int(round(y_txt+COORD_OFFSET))
        if 0<=y<26 and 0<=x<26:
            canvas[y][x]=AGENT_LABELS[idx]
    print(f'\n【{fname} | step={step}】')
    for row in canvas: print(''.join(row))

def load_and_show(path):
    with open(path,encoding='utf-8') as f:
        lines=f.read().splitlines()

    base_maze=[re.sub(r'[A-Z]',' ',r) for r in lines[1:27]]
    coords=parse_coords([ln for ln in lines[27:] if '[' in ln])

    if coords.size==0 or coords.shape[0]%N_AGENTS:
        print(f' {os.path.basename(path)} 不匹配 (T,{N_AGENTS},2)，跳過')
        return

    traj=coords.reshape(-1,N_AGENTS,2)

    if STEPS_TO_SHOW is None: steps=range(len(traj))
    elif isinstance(STEPS_TO_SHOW,int): steps=range(min(len(traj),STEPS_TO_SHOW))
    else: steps=[s for s in STEPS_TO_SHOW if 0<=s<len(traj)]

    for s in steps: draw_ascii(base_maze,traj[s],s,os.path.basename(path))

# ---------- main ----------
def main():
    if not DIR_TXT.is_dir():
        print('找不到資料夾:',DIR_TXT); return

    all_txt=sorted(
        f for f in os.listdir(DIR_TXT)
        if f.startswith(AGENT_PREFIX) and f.endswith('.txt') and 'FINAL' not in f
    )

    if FILES_TO_SHOW is None: targets=all_txt
    elif isinstance(FILES_TO_SHOW,int): targets=all_txt[:FILES_TO_SHOW]
    else: targets=[f for f in all_txt if f in FILES_TO_SHOW]

    if not targets:
        print('沒有符合條件的檔案可顯示'); return

    for fname in targets:
        load_and_show(DIR_TXT/fname)

if __name__=='__main__':
    main()


【sim1_1.txt | step=0】
##########################
#                        #
#                        #
#                        #
#                        #
#                        #
#     ##############     #
#     #            #     #
#     #            #     #
#     #            #     #
#     #     A      #     #
#     #            #     #
#     #     S      #     #
#     #            #     #
#     #            #     #
#     #            #     #
#     #            #     #
#     #     B      #     #
#     #            #     #
#     ##############     #
#                        #
#                        #
#                        #
#                        #
#                        #
##########################

【sim1_1.txt | step=1】
##########################
#                        #
#                        #
#                        #
#                        #
#                        #
#     ##############     #
#     #            #     #
#     #            #     #
#     #  

In [95]:
PROJECT_ROOT   = Path('/Users/Jer_ry/Desktop/scripts')
SIM_FOLDER     = FNAME
STEP_LIMIT     = 15 
COORD_OFFSET   = 6
COLOR_PALETTE  = [
    '#1f77b4','#ff7f0e','#2ca02c','#d62728','#9467bd',
    '#8c564b','#e377c2','#7f7f7f','#bcbd22','#17becf'
]
ALPHA_FILL     = .8
VIEW_X_MIN, VIEW_X_MAX = 7, 18
VIEW_Y_MIN, VIEW_Y_MAX = 7, 18
# ──────────────────────────────────

DATA_ROOT = PROJECT_ROOT/'data'/SIM_FOLDER       
PLOT_ROOT = DATA_ROOT                            

def min_factor_gt1(n:int)->int:
    if n < 2: return 1
    for d in range(2,int(math.sqrt(n))+2):
        if n % d == 0: return d
    return n

def parse_all_floats(lines_after_maze:list)->list[float]:
    return list(map(float, re.findall(r'-?\d+(?:\.\d+)?',
                                      ''.join(lines_after_maze))))

def draw_one_step(ax, pos, labels, colors):
    for (x,y),c,lbl in zip(pos, colors, labels):
        if VIEW_X_MIN<=x<=VIEW_X_MAX and VIEW_Y_MIN<=y<=VIEW_Y_MAX:
            ax.add_patch(plt.Rectangle((x-.5,y-.5),1,1,
                                       facecolor=c,edgecolor='black',
                                       alpha=ALPHA_FILL,lw=1.2))
            ax.text(x,y,lbl,ha='center',va='center',color='white',
                    weight='bold',fontsize=12)

def export_png(fig, out_path):
    fig.tight_layout()
    fig.savefig(out_path, dpi=150, bbox_inches='tight')
    plt.close(fig)

# ===== A. rulemap / random / intermediate =================================
def batch_plot_general(move_type:str, src_dir:Path, dst_dir:Path):
    txt_list = sorted(
        t for t in glob.glob(str(src_dir/'**'/'*.txt'), recursive=True)
        if 'FINAL' not in os.path.basename(t)
    )
    if not txt_list:
        print(f'CANT FIND：{src_dir}')
        return

    dst_dir.mkdir(parents=True, exist_ok=True)
    for txt in txt_list:
        with open(txt, encoding='utf-8') as f:
            lines = f.read().splitlines()

        letters = {ch for r in lines[1:27] for ch in r
                   if ch=='S' or 'A'<=ch<='Z'}
        if letters:
            N = len(letters)
        else:
            total_float = len(parse_all_floats(lines[27:]))
            N = min_factor_gt1(total_float//2)
        labels = [str(i+1) for i in range(N)]
        colors = list(itertools.islice(itertools.cycle(COLOR_PALETTE), N))

        floats = parse_all_floats(lines[27:])
        traj   = np.asarray(floats,dtype=float).reshape(-1,N,2)
        max_s  = len(traj) if STEP_LIMIT is None else min(len(traj),STEP_LIMIT+1)

        trial_id = Path(txt).stem
        out_trial = dst_dir/f'step_plot_{trial_id}'
        out_trial.mkdir(exist_ok=True)

        prev = None
        for s in range(max_s):
            pos = traj[s]
            if prev is not None and np.array_equal(pos,prev):
                continue
            prev = pos.copy()
            pos_plot = pos + COORD_OFFSET

            fig,ax = plt.subplots(figsize=(7,7))
            ax.set_facecolor('white')
            ax.set_xlim(VIEW_X_MIN-.5,VIEW_X_MAX+.5)
            ax.set_ylim(VIEW_Y_MAX+.5,VIEW_Y_MIN-.5)
            ax.set_aspect('equal')
            ax.set_xticks(np.arange(VIEW_X_MIN-.5,VIEW_X_MAX+1.5,1))
            ax.set_yticks(np.arange(VIEW_Y_MIN-.5,VIEW_Y_MAX+1.5,1))
            ax.grid(True,lw=.8,color='lightgrey')
            ax.tick_params(length=0,labelleft=False,labelbottom=False)

            draw_one_step(ax,pos_plot,labels,colors)
            # ax.set_title(f'{trial_id} – step {s}')
            export_png(fig,out_trial/f'step_{s:03d}.png')

        print(f'✓ {move_type:<12} {trial_id} → {out_trial}')

# ===== B. logic ===========================================================
ODD  = [1,3,5,7,9]
EVEN = [0,2,4,6,8]
LOGIC_COLOR = ['#1f77b4','#ff7f0e','#2ca02c']  

def get_parity(case:int)->dict[int,str]:
    grp = (case-1)//12 + 1
    if grp==1: return {0:'odd', 1:'odd', 2:'even'}
    if grp==2: return {0:'odd', 1:'even',2:'odd'}
    if grp==3: return {0:'even',1:'odd', 2:'odd'}
    raise ValueError('case_type 必須 1-36')

def batch_plot_logic(src_dir:Path, dst_dir:Path):
    txt_list = sorted(src_dir.glob('logic_case*.txt'))
    if not txt_list:
        print(f'CNAT FIND TXT：{src_dir}')
        return

    dst_dir.mkdir(parents=True, exist_ok=True)
    for txt in txt_list:
        case = int(re.search(r'case(\d+)', txt.stem).group(1))
        parity = get_parity(case)

        with open(txt,encoding='utf-8') as f:
            lines = f.read().splitlines()
        floats = parse_all_floats(lines[27:])
        traj   = np.asarray(floats,dtype=float).reshape(-1,3,2)
        max_s  = len(traj) if STEP_LIMIT is None else min(len(traj),STEP_LIMIT+1)

        trial_dir = dst_dir/f'step_plot_{txt.stem}'
        trial_dir.mkdir(exist_ok=True)

        for s in range(max_s):
            pos_plot = traj[s]+COORD_OFFSET
            fig,ax = plt.subplots(figsize=(7,7))
            ax.set_facecolor('white')
            ax.set_xlim(VIEW_X_MIN-.5,VIEW_X_MAX+.5)
            ax.set_ylim(VIEW_Y_MAX+.5,VIEW_Y_MIN-.5)
            ax.set_aspect('equal')
            ax.set_xticks(np.arange(VIEW_X_MIN-.5,VIEW_X_MAX+1.5,1))
            ax.set_yticks(np.arange(VIEW_Y_MIN-.5,VIEW_Y_MAX+1.5,1))
            ax.grid(True,lw=.8,color='lightgrey')
            ax.tick_params(length=0,labelleft=False,labelbottom=False)

            labels=[]
            for i in range(3):
                labels.append(str(random.choice(ODD if parity[i]=='odd' else EVEN)))
            draw_one_step(ax,pos_plot,labels,LOGIC_COLOR)
            # ax.set_title(f'{txt.stem} – step {s}')
            export_png(fig, trial_dir/f'step_{s:03d}.png')
        print(f'✓ logic        {txt.stem} → {trial_dir}')

# ===== run everything =====================================================
def main():
    # A-1 rulemap
    batch_plot_general('rulemap',
        DATA_ROOT/'rulemap',
        PLOT_ROOT/'rulemap_plot')

    # A-2 random
    batch_plot_general('random',
        DATA_ROOT/'random',
        PLOT_ROOT/'random_plot')

    # A-3 intermediate
    batch_plot_general('intermediate',
        DATA_ROOT/'intermediate',
        PLOT_ROOT/'intermediate_plot')

    # B   logic
    batch_plot_logic(
        DATA_ROOT/'logic',
        PLOT_ROOT/'logic_plot')

    print('\ncomple！\nplace：')
    for p in ['rulemap_plot','random_plot','intermediate_plot','logic_plot']:
        print('   ', PLOT_ROOT/p)

if __name__ == '__main__':
    main()

✓ rulemap      sim1_1 → /Users/Jer_ry/Desktop/scripts/data/sim1/rulemap_plot/step_plot_sim1_1
✓ rulemap      sim1_10 → /Users/Jer_ry/Desktop/scripts/data/sim1/rulemap_plot/step_plot_sim1_10
✓ rulemap      sim1_10 → /Users/Jer_ry/Desktop/scripts/data/sim1/rulemap_plot/step_plot_sim1_10
✓ rulemap      sim1_11 → /Users/Jer_ry/Desktop/scripts/data/sim1/rulemap_plot/step_plot_sim1_11
✓ rulemap      sim1_11 → /Users/Jer_ry/Desktop/scripts/data/sim1/rulemap_plot/step_plot_sim1_11
✓ rulemap      sim1_12 → /Users/Jer_ry/Desktop/scripts/data/sim1/rulemap_plot/step_plot_sim1_12
✓ rulemap      sim1_12 → /Users/Jer_ry/Desktop/scripts/data/sim1/rulemap_plot/step_plot_sim1_12
✓ rulemap      sim1_13 → /Users/Jer_ry/Desktop/scripts/data/sim1/rulemap_plot/step_plot_sim1_13
✓ rulemap      sim1_13 → /Users/Jer_ry/Desktop/scripts/data/sim1/rulemap_plot/step_plot_sim1_13
✓ rulemap      sim1_14 → /Users/Jer_ry/Desktop/scripts/data/sim1/rulemap_plot/step_plot_sim1_14
✓ rulemap      sim1_14 → /Users/Jer_ry/Des